In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
# printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [ ]:
news = pd.read_csv("D:/Machine learning/Fake News Prediction using Machine Learning with Python/news.csv")
news

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [10]:
news.shape

(6335, 4)

Ubah kolom 'label' menjadi <br>
1 : REAL <br>
2 : FAKE

In [ ]:
news['label'] = news['label'].replace({'REAL':1, 'FAKE': 0}) 

C:\Users\ACER\AppData\Local\Temp\ipykernel_7868\60646276.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news['label'] = news['label'].replace({'REAL':1, 'FAKE': 0})


In [5]:
news

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


Menghitung banyaknya missing values

In [12]:
news.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [13]:
# Jika ada data yang hilang bisa diisi dengan empty string
news = news.fillna('')

In [15]:
# Menggabungkan kolom title dan text
news['content'] = news['title'] + '' + news['text']

In [18]:
print(news['content'])


0       You Can Smell Hillary’s FearDaniel Greenfield,...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathyU.S...
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [19]:
# memisahkan kolom content dan kolom label
X = news.drop(columns='label', axis = 1)
Y = news['label']

In [21]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

Stemmning <br>
Stemming is the process of reducing a word to its Root words <br>
example : <br>
actor, actress, acting --> act

In [23]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content) # remove non alphabets (ex: numbers)
    stemmed_content = stemmed_content.lower() # convert all alphabets to lowercase
    stemmed_content = stemmed_content.split() # convert to list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content
    

In [26]:
news['content'] = news['content'].apply(stemming)

In [27]:
print(news['content'])

0       smell hillari feardaniel greenfield shillman j...
1       watch exact moment paul ryan commit polit suic...
2       kerri go pari gestur sympathyu secretari state...
3       berni support twitter erupt anger dnc tri warn...
4       battl new york primari mattersit primari day n...
                              ...                        
6330    state depart say find email clinton specialist...
6331    p pb stand plutocrat pentagon p pb stand pluto...
6332    anti trump protest tool oligarchi inform anti ...
6333    ethiopia obama seek progress peac secur east a...
6334    jeb bush suddenli attack trump mattersjeb bush...
Name: content, Length: 6335, dtype: object


In [28]:
#seperating the data and label
X = news['content'].values
Y = news['label'].values

In [29]:
print(X)

['smell hillari feardaniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scan

In [30]:
print(Y)

[0 0 1 ... 0 1 1]


In [31]:
Y.shape

(6335,)

In [32]:
X.shape

(6335,)

In [34]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [35]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1611127 stored elements and shape (6335, 48000)>
  Coords	Values
  (0, 106)	0.027454637927364703
  (0, 219)	0.023221866687915392
  (0, 283)	0.054029850061814134
  (0, 342)	0.06006120561496521
  (0, 381)	0.013871725953572337
  (0, 487)	0.020295962593590754
  (0, 625)	0.07884074031951002
  (0, 675)	0.01978505495781817
  (0, 678)	0.01862575483544748
  (0, 685)	0.04442179878453866
  (0, 716)	0.015435610422349796
  (0, 750)	0.01903921312235206
  (0, 1078)	0.026586859221488165
  (0, 1102)	0.020630944978050337
  (0, 1125)	0.03697738116152465
  (0, 1141)	0.01475870080775467
  (0, 1201)	0.028685669251996487
  (0, 1206)	0.01804241404226708
  (0, 1316)	0.02242620891400724
  (0, 1332)	0.010628725596044625
  (0, 1713)	0.02500720719242767
  (0, 1837)	0.02589549256753411
  (0, 1888)	0.04101373172769832
  (0, 1923)	0.027526964247794
  (0, 1964)	0.018801849839639295
  :	:
  (6334, 43538)	0.023427756006768236
  (6334, 43874)	0.039804178042262

Splitting the dataset to training and test data

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, stratify=Y, random_state=2)

Training the model : Logistic Regression

In [37]:
model = LogisticRegression()

In [38]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [39]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [40]:
print('Accuracyscore of the traing data : ', training_data_accuracy)

Accuracyscore of the traing data :  0.9534333070244673


In [41]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [43]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9179163378058406


Making a Predictive System

In [58]:
X_new = X_test[4]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('The news is Fake')
else:
    print('the news is Real')

[0]
The news is Fake


In [57]:
print(Y_test[4])

0
